In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
import time

In [34]:
# Example we are trying to scrape text from the blow url. Because Instagram
# requires login we suggest that you auto login using chrome and then use chrome user data
# as an argument to load the chrome browser
# The browser is fully loaded and viewable to the user as this is not a fully automated scraper
# Close any open chrome browser before running or an error is thrown
url = 'https://www.instagram.com/eggsoeufs/'
options = webdriver.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\alexe\AppData\Local\Google\Chrome\User Data")
options.add_argument(r'--profile-directory=Default')
driver = webdriver.Chrome(executable_path=r"chromedriver.exe", options=options)
driver.get(url)

In [118]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [35]:
# We will auto scroll until either we can't scroll anymore or the number of iterations of the scroll
# reaches 0 based on the count value. Thus if the page has a large number of inputs we will not be able
# to save everything.
# Since we are trying to get the text of the images we are collecting the links and storing them in list_links
last_height = driver.execute_script("return document.body.scrollHeight")
boolScroll = True
count = 100
set_links = set()
while (boolScroll) and (count > 0):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    count = count - 1
    temp = driver.find_elements(By.XPATH,"//div[@class='_aabd _aa8k  _al3l']")
    for item in temp:
        try:
            temp1 = item.find_element(By.TAG_NAME, 'a')
            set_links.add(temp1.get_attribute('href'))
        except StaleElementReferenceException as Exception:
            time.sleep(0.5)
    time.sleep(7)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        boolScroll = False
        print(new_height)
    else:
        last_height = new_height
    print(last_height)

3226
5417
7921
9173


In [36]:
print(len(set_links))

69


In [39]:
df = pd.DataFrame(columns = ['Title', 'Date', 'Text', 'Poster', 'MultiMedia'])
for link in set_links:
    driver.get(link)
    time.sleep(5)
    title = driver.find_element(By.XPATH, "//time[@class='_aaqe']").get_attribute("title")
    date = driver.find_element(By.XPATH,"//time[@class='_aaqe']").get_attribute("title")
    text = driver.find_element(By.XPATH, "//h1[@class='_aacl _aaco _aacu _aacx _aad7 _aade']").text
    by = driver.find_element(By.XPATH, "//div[@class='xt0psk2']").text
    mm_link = driver.find_element(By.XPATH, "//div[@class='_aagv']/img").get_attribute('src')
    df_temp = pd.DataFrame({'Title' : [title], 'Date' : [date], 'Text' : [text], 'Poster' : [by], 'MultiMedia' : [mm_link]})
    df = pd.concat([df,df_temp])
df.shape

(69, 5)

In [40]:
df.head()

,Title,Date,Text,Poster,MultiMedia
0,"Mar 10, 2023","Mar 10, 2023",Hey #Vancity! Have the best of both worlds at ...,eggsoeufs,https://scontent.cdninstagram.com/v/t51.2885-1...
0,"Mar 16, 2023","Mar 16, 2023",We need your vote! It’s time for #MarchMadness...,fosterfeasts,https://instagram.fymq2-1.fna.fbcdn.net/v/t51....
0,"Apr 21, 2023","Apr 21, 2023",Level up your traditional #GrilledCheese...tha...,eggsoeufs,https://scontent.cdninstagram.com/v/t51.2885-1...
0,"Mar 13, 2023","Mar 13, 2023","To kick off #MarchBreak, we've got an eggcelle...",eggsoeufs,https://scontent.cdninstagram.com/v/t51.2885-1...
0,"Mar 31, 2023","Mar 31, 2023",Shake things up and opt for #SavouryCrepes wit...,eggsoeufs,https://scontent.cdninstagram.com/v/t51.2885-1...


In [23]:
driver.get(list(set_links)[-1])

In [24]:
driver.find_element(By.XPATH,"//time[@class='_aaqe']").get_attribute("title")

'Aug 12, 2022'

In [25]:
driver.find_element(By.XPATH, "//h1[@class='_aacl _aaco _aacu _aacx _aad7 _aade']").text

'My weekends are truly not complete without #BrunchTime🍳 and my brunch meal staple is ALWAY EGGS 🍳. #ad I love all the creative ways our local restaurants use eggs in their menus, from a good old-fashioned classic breakfast to an unforgettable brunch bowl egg-sperience. Here are some of my favourite local spots featuring egg-ceptional egg dishes that you need to check out:\n\n🥚 Classic Breakfast from @bluestar_yyc\n\n🥚Bagel Forestiere from @oeb_breakfast\n\n🥚Truffle bacon mushroom bowl and Duck Confit Monte Cristo from @namoyyc\n\n🥚Bird nest from @zcrewcafe\n\nLet me know your favourite brunch spot in Calgary! @eggsoeufs #EggsAnytime #LocalLove'

In [26]:
driver.find_element(By.XPATH, "//div[@class='xt0psk2']").text

'rachelhaneats'

In [27]:
driver.find_element(By.XPATH, "//div[@class='_aagv']/img").get_attribute('src')

'https://scontent.cdninstagram.com/v/t51.2885-15/298670680_610408290444642_6670465811055762567_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent.cdninstagram.com&_nc_cat=106&_nc_ohc=GG5oz4w6OhsAX_yZklV&edm=APs17CUBAAAA&ccb=7-5&ig_cache_key=MjkwMzQwNjMzMDYyODcyMDM4Mg%3D%3D.2-ccb7-5&oh=00_AfDhg1fDDiA1tizvKV_NNGp_QkvIG5X71Hawd2FG_2k21w&oe=6466866E&_nc_sid=978cb9'